# Toronto Neighbourhoods

## Part 3: Segmenting and Clustering

In part 2, the list of Toronto neighbourhoods, which has been scraped from the Wikipedia page, has been enriched with location data. This list has been saved as a '.csv' file, which can now be loaded for further enrichment with data from Foursquare.

In [2]:
import pandas as pd
import numpy as np

import folium
import requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim # Nominatim is the open acces geolocation service that geopy uses.

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [3]:
path = '~/Documents/Projects/Coursera-Capstone/Neighbourhoods.csv'
toronto_nbhs = pd.read_csv(path)

# remove the 'index' column which is explicitly saved (and therefore unnamed) in the '.csv'
toronto_nbhs = toronto_nbhs[['Postal Code', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]


### Visualising the Toronto neighbourhoods

Start the analysis by creating a map of the Toronto neighbourhoods to get an idea for the spatial distribution.

In [4]:
# Obtain the coordinates of Toronto

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="coursera_capstone")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [5]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto_nbhs['Latitude'], toronto_nbhs['Longitude'], toronto_nbhs['Borough'], toronto_nbhs['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

### Obtaining venue information

For all the Toronto neighbourhoods, obtain all the venues within a certain range from Foursquare. Start by defining a function that automates the get request.

In [44]:
# function that extracts venue names, locations and categories from location data in a dataframe
def getNearbyVenues(names, latitudes, longitudes, radius=1500): # radius is 1500 m because Toronto appears to be less dense than Manhattan
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Store the Foursquare credentials in easily accessible variables.

In [45]:
CLIENT_ID = '....' # your Foursquare ID
CLIENT_SECRET = '....' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

Call the newly defined function to obtain the venue information for all Toronto neighbourhoods.

In [46]:
toronto_venues = getNearbyVenues(names=toronto_nbhs['Neighbourhood'],
                                   latitudes=toronto_nbhs['Latitude'],
                                   longitudes=toronto_nbhs['Longitude']
                                  )
print(toronto_venues.shape)
toronto_venues.head()

(6884, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
1,"Malvern, Rouge",43.806686,-79.194353,Harvey's,43.800020,-79.198307,Restaurant
2,"Malvern, Rouge",43.806686,-79.194353,Canadiana exhibit,43.817962,-79.193374,Zoo Exhibit
3,"Malvern, Rouge",43.806686,-79.194353,RBC Royal Bank,43.798782,-79.197090,Bank
4,"Malvern, Rouge",43.806686,-79.194353,Staples Morningside,43.800285,-79.196607,Paper / Office Supplies Store


Save the venues dataframe for safe keeping in a '.csv'.

In [47]:
# Save the cleaned dataframe as a '.csv'
path = '~/Documents/Projects/Coursera-Capstone/Toronto_venues.csv'

toronto_venues.to_csv(path)

### Visualise the data

Visualise the venues dataframe, grouped by neighourhood in order to get an idea of the number of venues that are returned for each neighbourhood. 

In [6]:
toronto_venues.groupby('Neighbourhood').count()

,Unnamed: 0,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,
Agincourt,62,62,62,62,62,62,62
"Alderwood, Long Branch",46,46,46,46,46,46,46
"Bathurst Manor, Wilson Heights, Downsview North",40,40,40,40,40,40,40
Bayview Village,16,16,16,16,16,16,16
"Bedford Park, Lawrence Manor East",82,82,82,82,82,82,82
...,...,...,...,...,...,...,...
"Willowdale, Willowdale West",39,39,39,39,39,39,39
Woburn,32,32,32,32,32,32,32
Woodbine Heights,69,69,69,69,69,69,69


In [49]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 349 uniques categories.


Note that there are 99 rows, i.e. there are 99 neighbourhoods that yield venue information.

### Analyse each neighbourhood

Use one hot encoding to map for each neighourhood whether or not it contains each venue category.

In [7]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]


In [8]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,...,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.021739,0.0,0.0,0.000000,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.025,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.012195,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
95,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
96,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
97,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.011364,0.0,0.0


Sort the venues for each neigbourhood in descending order from most often to leas often occuring.

In [12]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

temp = neighbourhoods_venues_sorted
neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Breakfast Spot,Park,Coffee Shop,Cantonese Restaurant,Caribbean Restaurant,Bakery,Hong Kong Restaurant,Gym / Fitness Center
1,"Alderwood, Long Branch",Pizza Place,Park,Café,Coffee Shop,Burger Joint,Grocery Store,Department Store,Bank,Bar,Bakery
2,"Bathurst Manor, Wilson Heights, Downsview North",Park,Coffee Shop,Pizza Place,Gas Station,Bank,Bridal Shop,Fried Chicken Joint,Gym,Mediterranean Restaurant,Men's Store
3,Bayview Village,Gas Station,Bank,Park,Trail,Japanese Restaurant,Intersection,Athletics & Sports,Café,Chinese Restaurant,Grocery Store
4,"Bedford Park, Lawrence Manor East",Bakery,Italian Restaurant,Sushi Restaurant,Coffee Shop,Bagel Shop,Pub,Pizza Place,Café,Sandwich Place,Bank


### Cluster neighbourhoods

In [14]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 0, 0, 2, 2, 0, 2, 2, 2], dtype=int32)

### Merge and clean neighbourhood, venue and cluster information.

First merge the venue dataframe that contains the top 10 venues for each neighbourhood with the cluster labels.

In [18]:
# add clustering labels
neighbourhoods_venues_sorted = temp
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# initialise the new dataframe with the neighbourhood names and locations
toronto_merged = toronto_nbhs

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

 

From 103 postal codes, only 99 neighbourhoods yield venue information and are therefore labeled for a cluster. Therefore, when merging with the neighbourhood information, some neighbourhoods will have the cluster label 'NaN'. This forces the other cluster labels to be cast as type 'float', which causes problems  for the visualisation. NaN, therefore must be replaced with an integer and the entire column must be recast as type 'integer'.

In [19]:
# NaN' cluster labels are replaced with value 5, cluster labels are then recast as type 'int'
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].fillna(5)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

toronto_merged.head() 

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1,Zoo Exhibit,Fast Food Restaurant,Pizza Place,Restaurant,Coffee Shop,Spa,Supermarket,Gas Station,Caribbean Restaurant,Other Great Outdoors
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,Park,Gym,Hotel,Grocery Store,Breakfast Spot,Gym / Fitness Center,Burger Joint,Italian Restaurant,Neighborhood,Noodle House
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Pizza Place,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Bank,Sandwich Place,Liquor Store,Gym / Fitness Center,Greek Restaurant,Park
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Indian Restaurant,Pharmacy,Supermarket,Fast Food Restaurant,Pizza Place,Convenience Store,Sandwich Place,Park,Chinese Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Restaurant,Coffee Shop,Bakery,Indian Restaurant,Gas Station,Sandwich Place,Clothing Store,Pharmacy,Cosmetics Shop,Bank


In [20]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Note that similar neighbourhoods in Toronto are, apparently, roughly geographically oriented somewhat concentrically around the harbour and old Toronto.

In [59]:
toronto_merged.groupby(['Cluster Labels']).count()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,,
0,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
1,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49
2,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39
3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


## View and describe the clusters

#### Cluster 0: Park and services (bank, gas station, pharmacy, etc.)

In [60]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Rouge Hill, Port Union, Highland Creek",0,Park,Gym,Hotel,Grocery Store,Breakfast Spot,Gym / Fitness Center,Burger Joint,Italian Restaurant,Neighborhood,Noodle House
8,"Cliffside, Cliffcrest, Scarborough Village West",0,Park,Harbor / Marina,Sandwich Place,Pub,Fast Food Restaurant,Ice Cream Shop,Pizza Place,Coffee Shop,Beach,Grocery Store
9,"Birch Cliff, Cliffside West",0,Park,Convenience Store,Thai Restaurant,Gym Pool,Gym,Golf Course,General Entertainment,Filipino Restaurant,Diner,College Stadium
19,Bayview Village,0,Gas Station,Bank,Park,Trail,Japanese Restaurant,Intersection,Athletics & Sports,Café,Chinese Restaurant,Grocery Store
28,"Bathurst Manor, Wilson Heights, Downsview North",0,Park,Coffee Shop,Pizza Place,Gas Station,Bank,Bridal Shop,Fried Chicken Joint,Gym,Mediterranean Restaurant,Men's Store
88,"New Toronto, Mimico South, Humber Bay Shores",0,Park,Indian Restaurant,American Restaurant,Grocery Store,Bakery,Café,Pharmacy,Coffee Shop,Bar,Convenience Store
91,"Old Mill South, King's Mill Park, Sunnylea, Hu...",0,Park,Coffee Shop,Italian Restaurant,Pizza Place,Restaurant,Sandwich Place,Bank,Sushi Restaurant,Beer Store,Fast Food Restaurant
93,"Islington Avenue, Humber Valley Village",0,Pharmacy,Bank,Park,Playground,Bakery,Shopping Mall,Golf Course,Café,Liquor Store,Convenience Store
94,"West Deane Park, Princess Gardens, Martin Grov...",0,Park,Sandwich Place,Hotel,Fish & Chips Shop,Pharmacy,Coffee Shop,Pizza Place,Restaurant,Gym,Gym / Fitness Center
96,Humber Summit,0,Park,Bakery,Asian Restaurant,ATM,Pharmacy,Sports Bar,Food Court,Mexican Restaurant,Café,Skating Rink


#### Custer 1: Coffee shops and other hospitality

In [61]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Malvern, Rouge",1,Zoo Exhibit,Fast Food Restaurant,Pizza Place,Restaurant,Coffee Shop,Spa,Supermarket,Gas Station,Caribbean Restaurant,Other Great Outdoors
2,"Guildwood, Morningside, West Hill",1,Pizza Place,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Bank,Sandwich Place,Liquor Store,Gym / Fitness Center,Greek Restaurant,Park
3,Woburn,1,Coffee Shop,Indian Restaurant,Pharmacy,Supermarket,Fast Food Restaurant,Pizza Place,Convenience Store,Sandwich Place,Park,Chinese Restaurant
4,Cedarbrae,1,Restaurant,Coffee Shop,Bakery,Indian Restaurant,Gas Station,Sandwich Place,Clothing Store,Pharmacy,Cosmetics Shop,Bank
5,Scarborough Village,1,Sandwich Place,Pharmacy,Ice Cream Shop,Bank,Wings Joint,Coffee Shop,Breakfast Spot,Pizza Place,Chinese Restaurant,Grocery Store
6,"Kennedy Park, Ionview, East Birchmount Park",1,Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Pizza Place,Sandwich Place,Pharmacy,Discount Store,Grocery Store,Bus Stop,Beer Store
7,"Golden Mile, Clairlea, Oakridge",1,Coffee Shop,Pizza Place,Bakery,Sandwich Place,Hardware Store,Convenience Store,Mexican Restaurant,Fast Food Restaurant,Beer Store,Trail
10,"Dorset Park, Wexford Heights, Scarborough Town...",1,Coffee Shop,Fast Food Restaurant,Indian Restaurant,Restaurant,Park,Pizza Place,Chinese Restaurant,Bank,Light Rail Station,Pet Store
11,"Wexford, Maryvale",1,Middle Eastern Restaurant,Grocery Store,Pizza Place,Asian Restaurant,Rental Car Location,Intersection,Coffee Shop,Chinese Restaurant,Restaurant,Burger Joint
13,"Clarks Corners, Tam O'Shanter, Sullivan",1,Fast Food Restaurant,Pizza Place,Bank,Chinese Restaurant,Pharmacy,Coffee Shop,Cantonese Restaurant,Vietnamese Restaurant,Gas Station,Sandwich Place


#### Cluster 2: Coffee shops and

In [62]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,"East Toronto, Broadview North (Old East York)",2,Greek Restaurant,Café,Coffee Shop,Ice Cream Shop,Park,Thai Restaurant,Pizza Place,Bakery,Ethiopian Restaurant,Beer Bar
41,"The Danforth West, Riverdale",2,Greek Restaurant,Café,Park,Coffee Shop,Bakery,Pizza Place,Pub,Grocery Store,Vietnamese Restaurant,Yoga Studio
42,"India Bazaar, The Beaches West",2,Park,Coffee Shop,Indian Restaurant,Grocery Store,Café,Bakery,Brewery,Pub,Restaurant,Beach
43,Studio District,2,Coffee Shop,Café,Bar,Bakery,Park,Diner,Vietnamese Restaurant,Pub,Grocery Store,Pizza Place
45,Davisville North,2,Coffee Shop,Italian Restaurant,Park,Restaurant,Sushi Restaurant,Deli / Bodega,Bakery,Indian Restaurant,Café,Thai Restaurant
46,"North Toronto West, Lawrence Park",2,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Restaurant,Japanese Restaurant,Bakery,Fast Food Restaurant,Park,Sporting Goods Shop
47,Davisville,2,Italian Restaurant,Coffee Shop,Bakery,Pizza Place,Café,Indian Restaurant,Park,Restaurant,Japanese Restaurant,Sushi Restaurant
48,"Moore Park, Summerhill East",2,Italian Restaurant,Park,Coffee Shop,Sushi Restaurant,Grocery Store,Restaurant,Gym,Liquor Store,Café,Spa
49,"Summerhill West, Rathnelly, South Hill, Forest...",2,Italian Restaurant,Café,Coffee Shop,Park,Sushi Restaurant,Bank,Liquor Store,French Restaurant,Burger Joint,Spa
50,Rosedale,2,Park,Italian Restaurant,Coffee Shop,Grocery Store,Café,Indian Restaurant,Sushi Restaurant,French Restaurant,Gourmet Shop,Spa


#### Cluster 3: Asian restaurants

In [63]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Agincourt,3,Chinese Restaurant,Shopping Mall,Breakfast Spot,Park,Coffee Shop,Cantonese Restaurant,Caribbean Restaurant,Bakery,Hong Kong Restaurant,Gym / Fitness Center
14,"Milliken, Agincourt North, Steeles East, L'Amo...",3,Chinese Restaurant,Dessert Shop,Korean Restaurant,Coffee Shop,Intersection,Vietnamese Restaurant,Pizza Place,Pharmacy,Noodle House,Bakery
21,"Willowdale, Newtonbrook",3,Korean Restaurant,Coffee Shop,Café,Bubble Tea Shop,Restaurant,Shopping Mall,Bank,Gas Station,Japanese Restaurant,Middle Eastern Restaurant
22,"Willowdale, Willowdale East",3,Korean Restaurant,Bubble Tea Shop,Coffee Shop,Grocery Store,Ramen Restaurant,Japanese Restaurant,Pizza Place,Sushi Restaurant,Café,Sandwich Place


#### Cluster 4: Non-hospitality country oriented


In [64]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Upper Rouge,4,Donut Shop,National Park,Farm,Nightclub,Outdoor Supply Store,Other Great Outdoors,Organic Grocery,Optical Shop,Office,Nudist Beach


#### Cluster 5

In [65]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
